In [2]:
fname = "04_test.txt"

"04_test.txt"

In [20]:
function parse_file(fname)
    out = []
    nrows = 0
    ncols = 0
    open(fname) do f
        for l in eachline(f)
            cur_l = []
            for c in l
                append!(cur_l, c == '@')
            end
            push!(out, cur_l)
            nrows += 1
            if ncols == 0
                ncols = length(cur_l)
            end
        end
        
    end
    return (out, nrows, ncols)
end

parse_file (generic function with 1 method)

In [29]:
function count_available(fname)
    (m, nrows, ncols) = parse_file(fname)
    nb_avail = 0
    for i in range(2,nrows-1)
        for j in range(2,ncols-1)
            if ((m[i][j]) && 
                (m[i-1][j-1] + m[i-1][j] + m[i-1][j+1] + m[i+1][j-1] + m[i+1][j] + m[i+1][j+1] + m[i][j-1] + m[i][j+1] <= 3))
                nb_avail += 1
                #println("Found one at position (", i, ", ", j, ")")
            end
        end
    end
    for i in range(2,nrows-1)
        # j = 1
        if ((m[i][1]) && 
            (m[i-1][1] + m[i-1][2] + m[i+1][1] + m[i+1][2] + m[i][2] <= 3))
            nb_avail += 1
            #println("Found one at position (", i, ", 1)")
        end
        if ((m[i][ncols]) && 
            (m[i-1][ncols] + m[i-1][ncols-1] + m[i+1][ncols] + m[i+1][ncols-1] + m[i][ncols-1] <= 3))
            nb_avail += 1
            #println("Found one at position (", i, ", ", ncols, ")")
        end
    end
    for j in range(2,ncols-1)
        if ((m[1][j]) && 
            (m[2][j-1] + m[2][j] + m[2][j+1] + m[1][j-1] + m[1][j+1] <= 3))
            nb_avail += 1
            #println("Found one at position (1, ", j, ")")
        end
        if ((m[nrows][j]) && 
            (m[nrows-1][j-1] + m[nrows-1][j] + m[nrows-1][j+1] + m[nrows][j-1] + m[nrows][j+1] <= 3))
            nb_avail += 1
            #println("Found one at position (", nrows, ", ", j, ")")
        end
    end
    nb_avail += m[1][1] + m[1][ncols] + m[nrows][1] + m[nrows][ncols]
    
    return nb_avail
end

count_available (generic function with 1 method)

In [30]:
count_available(fname)

1389

In [31]:
fname = "04_1.txt"

"04_1.txt"

In [32]:
count_available(fname)

1389

In [33]:
using DataStructures

In [50]:
function count_unwrap(fname)
    (m, nrows, ncols) = parse_file(fname)
    nb_avail = 0
    available = Deque{Tuple}()
    # rolls = Deque{Tuple}()
    neighbours = [[0 for i in range(1,ncols)] for j in range(1,nrows)]
    for i in range(2,nrows-1)
        for j in range(2,ncols-1)
            if (m[i][j])
                nb_neighbours = m[i-1][j-1] + m[i-1][j] + m[i-1][j+1] + m[i+1][j-1] + m[i+1][j] + m[i+1][j+1] + m[i][j-1] + m[i][j+1]
                if (nb_neighbours <= 3)
                    push!(available, (i,j))
                    #println("Found one at position (", i, ", ", j, ")")
                else
                    # push!(rolls, (i,j))
                    neighbours[i][j] = nb_neighbours
                end
            end
        end
    end
    for i in range(2,nrows-1)
        # j = 1
        if (m[i][1]) 
            nb_neighbours = m[i-1][1] + m[i-1][2] + m[i+1][1] + m[i+1][2] + m[i][2]
            if ( nb_neighbours <= 3)
                push!(available, (i,1))
                #println("Found one at position (", i, ", 1)")
            else
                neighbours[i][1] = nb_neighbours
                # push!(rolls, (i,1))
            end
        end
        if (m[i][ncols])
            nb_neighbours = m[i-1][ncols] + m[i-1][ncols-1] + m[i+1][ncols] + m[i+1][ncols-1] + m[i][ncols-1]
            if (nb_neighbours <= 3)
                push!(available, (i,ncols))
                #println("Found one at position (", i, ", ", ncols, ")")
            else
                neighbours[i][ncols] = nb_neighbours
                # push!(rolls, (i,ncols))
            end
        end
    end
    for j in range(2,ncols-1)
        if (m[1][j])
            nb_neighbours = m[2][j-1] + m[2][j] + m[2][j+1] + m[1][j-1] + m[1][j+1]
            if (nb_neighbours <= 3)
                push!(available, (1,j))
                #println("Found one at position (1, ", j, ")")
            else
                neighbours[1][j] = nb_neighbours
                # push!(rolls, (1,j))
            end
        end
        if (m[nrows][j])
            nb_neighbours = m[nrows-1][j-1] + m[nrows-1][j] + m[nrows-1][j+1] + m[nrows][j-1] + m[nrows][j+1]
            if (nb_neighbours <= 3)
                push!(available, (nrows,j))
                #println("Found one at position (", nrows, ", ", j, ")")
            else
                neighbours[nrows][j] = nb_neighbours
                # push!(rolls, (nrows,j))
            end
        end
    end
    for (i,j) in [(1,1), (1,ncols), (nrows,1), (nrows, ncols)]
        if m[i][j]
            push!(available, (i,j))
        end
    end

    while(!isempty(available) )
        nb_avail += 1
        (i,j) = pop!(available)
        for (di,dj) in [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
            ni = i+di
            nj = j+dj
            if (ni > 0) && (ni <= nrows) && (nj > 0) && (nj <= ncols)
                if m[ni][nj] && (neighbours[ni][nj] > 3)
                    neighbours[ni][nj] -= 1
                    if neighbours[ni][nj] == 3
                        push!(available, (ni,nj))
                    end
                end
            end
        end
    end
    
    
    
    return nb_avail
end

count_unwrap (generic function with 1 method)

In [51]:
fname = "04_test.txt"

"04_test.txt"

In [52]:
count_unwrap(fname)

43

In [49]:
fname = "04_1.txt"
count_unwrap(fname)

9000